In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

# Load your dataset
filename = "NF-BOT-IOT"
train_data = pd.read_csv('/content/drive/MyDrive/dataset/NF-BOT-IOT_train_preprocessed.csv')
test_data = pd.read_csv('/content/drive/MyDrive/dataset/NF-BOT-IOT_test_preprocessed.csv')

# Prepare training data
X_train = train_data.drop(columns=['label'], axis=1)
y_train = train_data['label']

# Prepare test data
X_test = test_data.drop(columns=['label'], axis=1)
y_test = test_data['label']

# Define custom Weighted Superposition Attraction (WSA) algorithm for feature selection
class WSAFeatureSelection:
    def __init__(self, num_agents=2, max_iter=2):
        self.num_agents = num_agents
        self.max_iter = max_iter

    def fit(self, X, y):
        start_time = time.time()  # Start the timer
        num_features = X.shape[1]
        best_feature_set = None
        best_accuracy = 0.0

        for iteration in range(self.max_iter):
            agent_solutions = []

            for agent in range(self.num_agents):
                # Initialize the agent's feature subset randomly or using some heuristic
                selected_features = np.random.choice(num_features, size=np.random.randint(1, num_features), replace=False)

                # Evaluate subset using a simple model (e.g., Random Forest)
                clf = RandomForestClassifier()
                X_subset = X.iloc[:, selected_features]
                clf.fit(X_subset, y)
                accuracy = accuracy_score(y, clf.predict(X_subset))

                agent_solutions.append((selected_features, accuracy))

                # Update best solution found
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_feature_set = selected_features

            # Update feature subsets based on attractiveness (e.g., probability of selection)
            # You need to implement this part

        end_time = time.time()  # End the timer
        execution_time = end_time - start_time  # Calculate execution time

        self.selected_features = best_feature_set
        self.execution_time = execution_time
        self.num_selected_features = len(best_feature_set) if best_feature_set is not None else 0  # Number of selected features
        return self


# Instantiate and run the WSA algorithm for feature selection on training data
wsa = WSAFeatureSelection(num_agents=5, max_iter=10)
wsa.fit(X_train, y_train)

# Apply selected features to training data
selected_features_train = X_train.columns[wsa.selected_features]

# Save selected features of the training data to a CSV file
feature_name_train = filename + "_WSA_feature_train.csv"
pd.Series(selected_features_train).to_csv(feature_name_train, index=False, header=True)

# Create a DataFrame for optimization results
optimization_results = pd.DataFrame({
    "Optimization": ["WSA"],
    "Execution Time of Optimizer": [wsa.execution_time],
    "No of Feature Selected": [wsa.num_selected_features],
    "Selected Feature": [', '.join(selected_features_train.tolist())]
})

# Save optimization results to a CSV file
optimization_results.to_csv('NF-BOT-IOT_WSA_feature.csv', index=False)

